<a href="https://colab.research.google.com/github/Gofurjon01/NLP-Text-classification-/blob/main/NLP_(Text_classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Natural Language Processing (NLP) and Text Classification With Python

In [9]:
dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"

In [10]:
import requests
import zipfile

def get_data(url, fn):
    response = requests.get(url)
    f = open(fn,'wb')
    f.write(response.content)
    f.close

def extract_data(fn):
    with zipfile.ZipFile(fn, 'r') as zf:
        zf.extractall()

In [11]:
dataset_file_name = 'dataset.zip'
get_data(dataset_url, dataset_file_name)
extract_data(dataset_file_name)

In [12]:
import os

def rename_data_folder(fn, new_name):
  if os.path.exists(fn):
    os.rename(fn,new_name)

In [13]:
data_set_folder = 'sentiment labelled sentences'
rename_data_folder(data_set_folder, 'data')

In [14]:
import pandas as pd

filepath_dict = {
    'yelp': 'data/yelp_labelled.txt',
    'amazon': 'data/amazon_cells_labelled.txt',
    'imdb': 'data/imdb_labelled.txt'
}

df_list = []

for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source
    df_list.append(df)

df = pd.concat(df_list)
df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [16]:
sentences = ['John likes ice cream', 'John hates chocolate']

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

sentences = ['John likes ice cream', 'John hates chocolate']

# Set min_df to 1 to include all terms
vectorizer = CountVectorizer(min_df=1, lowercase=False)
vectorizer.fit(sentences)
vectorizer.vocabulary_

{'John': 0, 'likes': 5, 'ice': 4, 'cream': 2, 'hates': 3, 'chocolate': 1}

In [22]:
vectorizer.transform(sentences).toarray()

array([[1, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0]])

In [23]:
df_yelp = df[df['source'] == 'yelp']

sentences = df_yelp['sentence'].values
labels = df_yelp['label'].values

In [25]:
from sklearn.model_selection import train_test_split

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.25, random_state=1000)

In [26]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

x_train = vectorizer.transform(sentences_train)
x_test = vectorizer.transform(sentences_test)
x_train

<750x1714 sparse matrix of type '<class 'numpy.int64'>'
	with 7368 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_test, y_test)

print("Accuracy:", score)

Accuracy: 0.796


In [38]:
for source in df['source'].unique():
  df_source = df[df['source'] == source]
  sentences = df_source['sentence'].values
  labels = df_source['label'].values

  sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.25, random_state=1000)

  vectorizer = CountVectorizer()
  vectorizer.fit(sentences_train)
  x_train = vectorizer.transform(sentences_train)
  x_test = vectorizer.transform(sentences_test)

  classifier = LogisticRegression(C=100, max_iter=1000)
  classifier.fit(x_train, y_train)
  score = classifier.score(x_test, y_test)
  print('Accuracy for {} data: {:.4f}'.format(source, score))

Accuracy for yelp data: 0.7840
Accuracy for amazon data: 0.8040
Accuracy for imdb data: 0.7647
